In [ ]:
import dataclasses as dc
import uuid

In [ ]:
import pendulum as pdt
import parsy

In [ ]:
auto_pick_output_text = r"""INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'
INFO:root:len(native_project.ObservationSets.Items)=2
INFO:root:observation_set.Name='ParentWellObservations'
INFO:root:len(observation_set.GetObservations())=17
INFO:root:observation_set.Name='Auto-picked Observation Set3'
INFO:root:len(observation_set.GetObservations())=120
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.999.ifrac"
"""

In [ ]:
auto_pick_output_text

In [ ]:
# Utility parsers
colon = parsy.string(':') << parsy.whitespace.many()
comma = parsy.string(',') << parsy.whitespace.many()
dot = parsy.string('.')
equals = parsy.string('=')
greater_than = parsy.string('>')
hex_literal = parsy.string('0x') >> parsy.regex(r'[\da-fA-F]+').map(lambda hds: int(hds, 16))
integer = parsy.regex(r'\d+').map(int)
newline = parsy.string('\n')
left_brace = parsy.string('{')
left_paren = parsy.string('(')
less_than = parsy.string('<')
right_brace = parsy.string('}')
right_paren = parsy.string(')')
single_quote = parsy.string("'")

In [ ]:
comma.parse(",")

In [ ]:
comma.parse(", ")

In [ ]:
# Single-line parsers
single_quoted_text = (single_quote >> parsy.regex(r"[^']+") << single_quote)

auto_picked_observation_set = parsy.string("INFO:root:observation_set.Name='Auto-picked Observation Set3'")
parent_well_observations = parsy.string("INFO:root:observation_set.Name='ParentWellObservations'")
get_observations = parsy.string("INFO:root:len(observation_set.GetObservations())=") >> parsy.regex(r'\d+').map(int)
observation_set_items = parsy.string("INFO:root:len(native_project.ObservationSets.Items)=2")
oid_parser = parsy.string('UUID') >> left_paren >> single_quoted_text.map(uuid.UUID) << right_paren
output_path_name = (parsy.string('INFO:root:Wrote changes to') >>
                    parsy.regex(r' "c:\\src\\Orchid.IntegrationTestData\\frankNstein_Bakken_UTM13_FEET.\d{3}.ifrac"'))
project_name = parsy.string("INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'")
unique_attribute_count_per_stage_per_well_equals = parsy.string(
    r'INFO:root:Unique counts of attributes per stage per well=')
python_var_name = parsy.regex(r'[\w_\d]+')
python_attribute_name = (python_var_name << dot.optional()).many().map(lambda ns: '.'.join(ns))

In [ ]:
@parsy.generate
def attribute_count_per_stage_per_well():
    yield unique_attribute_count_per_stage_per_well_equals
    yield left_brace
    attribute_per_stage_per_well_count = yield integer
    yield right_brace
    
    return attribute_per_stage_per_well_count

In [ ]:
@parsy.generate
def get_second_observations_count():
    yield project_name
    yield newline >> observation_set_items
    yield newline >> parent_well_observations
    yield newline >> get_observations
    yield newline >> auto_picked_observation_set
    get_observations_count = yield newline >> get_observations
    yield (newline >> attribute_count_per_stage_per_well).optional()
    yield newline >> output_path_name
    yield newline

    return get_observations_count

In [ ]:
get_second_observations_count.parse(auto_pick_output_text)

In [ ]:
auto_pick_and_create_attributes_output_text = r"""INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'
INFO:root:len(native_project.ObservationSets.Items)=2
INFO:root:observation_set.Name='ParentWellObservations'
INFO:root:len(observation_set.GetObservations())=17
INFO:root:observation_set.Name='Auto-picked Observation Set3'
INFO:root:len(observation_set.GetObservations())=120
INFO:root:Unique counts of attributes per stage per well={2}
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.998.ifrac"
"""

In [ ]:
@parsy.generate
def get_attribute_count_for_each_stage_and_well():
    yield project_name
    yield newline >> observation_set_items
    yield newline >> parent_well_observations
    yield newline >> get_observations
    yield newline >> auto_picked_observation_set
    yield newline >> get_observations
    attribute_count_for_each_stage_and_well = yield newline >> attribute_count_per_stage_per_well
    yield newline >> output_path_name
    yield newline

    return attribute_count_for_each_stage_and_well

In [ ]:
get_attribute_count_for_each_stage_and_well.parse(auto_pick_and_create_attributes_output_text)

In [ ]:
add_stages_output_text = r"""INFO:root:CreatedStageDetails(name='Stage-36', shmin='8144.498 psi', cluster_count=0, global_stage_sequence_no=0, start_time='2018-06-06T05:34:03.684000+00:00', stop_time='2018-06-06T07:19:35.560000+00:00')
INFO:root:CreatedStageDetails(name='Stage-37', shmin='2.322 psi', cluster_count=0, global_stage_sequence_no=0, start_time='2018-06-15T14:11:40.450000+00:00', stop_time='2018-06-15T15:10:11.200000+00:00')
INFO:root:CreatedStageDetails(name='Stage-38', shmin='8041.893 psi', cluster_count=7, global_stage_sequence_no=0, start_time='2018-06-28T23:35:54.379000+00:00', stop_time='2018-06-29T01:18:05.840000+00:00')
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.996.ifrac"
"""

In [ ]:
@parsy.generate
def key_value_pair():
    yield python_var_name.desc('key')
    value = yield equals >> (single_quoted_text | integer)
    return value

In [ ]:
@dc.dataclass
class AddedStageDetails:
    stage_name: str
    shmin: str
    cluster_count: int
    global_stage_sequence_no: int
    start_time: str
    stop_time: str

@parsy.generate
def added_stage_details():
    yield parsy.string('INFO:root:CreatedStageDetails')
    yield left_paren
    stage_name = yield key_value_pair
    shmin = yield comma >> key_value_pair
    cluster_count = yield comma >> key_value_pair
    global_stage_sequence_no = yield comma >> key_value_pair
    start_time = yield (comma >> key_value_pair).map(pdt.parse).desc('start_time')
    stop_time = yield (comma >> key_value_pair).map(pdt.parse).desc('stop_time')
    yield right_paren

    return AddedStageDetails(stage_name=stage_name, shmin=shmin, cluster_count=cluster_count,
                             global_stage_sequence_no=global_stage_sequence_no,
                             start_time=start_time, stop_time=stop_time)

In [ ]:
key_value_pair.parse("name='Stage-36'")

In [ ]:
key_value_pair.parse('cluster_count=0')

In [ ]:
key_value_pair.parse("start_time='2018-06-06T05:34:03.684000+00:00'")

In [ ]:
add_stages_output_text.split('\n')[0]

In [ ]:
added_stage_details.parse(add_stages_output_text.split('\n')[0])

In [ ]:
@parsy.generate
def get_added_stages():
    added_stages_details = yield (added_stage_details << newline).at_least(1)
    yield output_path_name
    yield newline

    return added_stages_details

In [ ]:
get_added_stages.parse(add_stages_output_text)

In [ ]:
monitor_time_series_output_text = r"""
All time series in project
{UUID('07d3cc41-1040-4125-8e2e-71726a124181'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAAC460>,
 UUID('1b544d11-55bb-4058-a527-7368046045ec'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAAC610>,
 UUID('d46ce56b-db33-488f-901f-8ec060a4455c'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAAC760>,
 UUID('ed590275-68e9-4001-8934-40386dfd1472'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAA9C40>}

All monitors in project
{UUID('14607f23-95f4-4405-b34b-daa0f924c2be'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB2B20>,
 UUID('182fa5d0-5695-40e8-ad59-ed18e796ee9c'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB27C0>,
 UUID('4116e3d3-b1ba-4063-b41e-467c5c00eb20'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAA9130>,
 UUID('44e7ad1c-f6b9-411c-84c3-fa903b1a516c'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB2EE0>,
 UUID('5b68d8c4-a578-44e7-bc08-b1d83483c4ec'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB2310>,
 UUID('5e51285b-6ac9-4a23-a360-f56399e4fe6b'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB2D90>,
 UUID('6777b2fe-7575-4fed-a82a-bb0b0085152d'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB2C40>,
 UUID('6b024601-ef74-4a82-ae4a-2a91648cae07'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB62B0>,
 UUID('8660a506-e2a3-4427-8a03-d20e60c214df'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB2580>,
 UUID('8fab7763-8cad-42f4-8d44-899f2e8691bc'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB2550>,
 UUID('9d702765-5696-4b38-a54c-84813898f907'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB26A0>,
 UUID('be89b07b-e37f-4222-9759-acd5682dc7a0'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB2A00>,
 UUID('c1d35d86-a8a1-4e46-a303-f2f1011a399f'): <orchid.native_monitor_adapter.NativeMonitorAdapter object at 0x000001915BAB28E0>}

Monitor of interest:
  - Object ID: 5b68d8c4-a578-44e7-bc08-b1d83483c4ec
  - Display Name: Demo_2H - stage 1

Object ID of monitor time series of interest: 07d3cc41-1040-4125-8e2e-71726a124181

Head of time series
2018-05-27 18:46:21+00:00    13.21247
2018-05-27 18:47:18+00:00    13.25400
2018-05-27 18:47:48+00:00    13.28520
2018-05-27 18:48:18+00:00    13.26438
2018-05-27 18:48:48+00:00    13.24896
Name: MonitorData-Demo_2H, dtype: float64
"""

In [ ]:
oid_parser = parsy.string('UUID') >> left_paren >> single_quoted_text.map(uuid.UUID) << right_paren

In [ ]:
uuid_element = "UUID('07d3cc41-1040-4125-8e2e-71726a124181')"

In [ ]:
oid_parser.parse(uuid_element)

In [ ]:
time_series_adapter_output = '<orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAAC460>'

In [ ]:
time_series_adapter = less_than >> python_attribute_name << parsy.string(' object at ') << hex_literal << greater_than

In [ ]:
time_series_adapter.parse(time_series_adapter_output)

In [ ]:
@dc.dataclass
class BriefOrchidObject:
    object_id: uuid.UUID
    class_name: str


@parsy.generate
def brief_orchid_object():
    oid = yield oid_parser
    yield colon
    class_name = yield (less_than >> python_attribute_name
                        << parsy.string(' object at ')
                        << hex_literal
                        << greater_than)

    return BriefOrchidObject(object_id=oid, class_name=class_name)

In [ ]:
time_series_in_project_output = "UUID('07d3cc41-1040-4125-8e2e-71726a124181'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAAC460>"

In [ ]:
brief_orchid_object.parse(time_series_in_project_output)

In [ ]:
many_times_series_in_project_output = r"""{UUID('07d3cc41-1040-4125-8e2e-71726a124181'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAAC460>,
 UUID('1b544d11-55bb-4058-a527-7368046045ec'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAAC610>,
 UUID('d46ce56b-db33-488f-901f-8ec060a4455c'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAAC760>,
 UUID('ed590275-68e9-4001-8934-40386dfd1472'): <orchid.native_time_series_adapter.NativeTimeSeriesAdapter object at 0x000001915BAA9C40>}"""

In [ ]:
@parsy.generate
def brief_orchid_objects():
    yield parsy.whitespace.optional()
    yield left_brace
    brief_objects = yield (brief_orchid_object << comma.optional()).many()
    yield right_brace

    return brief_objects

In [ ]:
brief_orchid_objects.parse(many_times_series_in_project_output)

In [ ]:
@parsy.generate
def all_times_series_in_project():
    yield newline
    yield parsy.string('All time series in project')
    yield newline
    brief_objects = yield brief_orchid_objects

    return brief_objects

In [ ]:
def sections(all_output):
    return all_output.split('\n\n')

In [ ]:
all_times_series_in_project_output = sections(monitor_time_series_output_text)[0]

In [ ]:
all_times_series_in_project_output

In [ ]:
all_times_series_in_project.parse(all_times_series_in_project_output)

In [ ]:
all_monitors_in_project_output = sections(monitor_time_series_output_text)[1]
all_monitors_in_project_output

In [ ]:
@parsy.generate
def all_monitors_in_project():
    yield parsy.string('All monitors in project')
    yield newline
    brief_objects = yield brief_orchid_objects

    return brief_objects

In [ ]:
all_monitors_in_project.parse(all_monitors_in_project_output)